In [52]:
import sqlite3
import pandas as pd

In [53]:
con = sqlite3.connect(':memory:')

In [54]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [55]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [56]:
pd.read_sql(
    '''
    SELECT * FROM Customers
    ''',
    con,
)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,5023,Mexico
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
4,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden
...,...,...,...,...,...,...,...
86,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland
87,88,Wellington Importadora,Paula Parente,"Rua do Mercado, 12",Resende,08737-363,Brazil
88,89,White Clover Markets,Karl Jablonski,305 - 14th Ave. S. Suite 3B,Seattle,98128,USA
89,90,Wilman Kala,Matti Karttunen,Keskuskatu 45,Helsinki,21240,Finland


In [57]:
tasks = 0

In [58]:
#1.Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address    
    FROM Customers                           
    where Country in ('Germany', 'France') or City = 'Madrid'  
    order by Country
    ''',
    con,
)

,CustomerName,Country,Address
0,Blondel père et fils,France,"24, place Kléber"
1,Bon app',France,"12, rue des Bouchers"
2,Du monde entier,France,"67, rue des Cinquante Otages"
3,Folies gourmandes,France,"184, chaussée de Tournai"
4,France restauration,France,"54, rue Royale"
5,La corne d'abondance,France,"67, avenue de l'Europe"
6,La maison d'Asie,France,1 rue Alsace-Lorraine
7,Paris spécialités,France,"265, boulevard Charonne"
8,Spécialités du monde,France,"25, rue Lauriston"
9,Victuailles en stock,France,"2, rue du Commerce"


In [59]:
#2.Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

tasks+=1
pd.read_sql(
    '''
    SELECT country, COUNT(CustomerName) AS customer_count
    FROM customers
    GROUP BY country
    ORDER BY customer_count DESC
    LIMIT 3;
    ''',
    con,
)

,Country,customer_count
0,USA,13
1,Germany,11
2,France,11


In [60]:
#3.Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

tasks+=1
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate
    FROM (
    SELECT ShipperName, OrderDate, ROW_NUMBER() OVER (ORDER BY OrderDate) AS row_num
    FROM Shippers S left join Orders O on S.ShipperID=O.ShipperID
    ) AS ordered_orders
    WHERE row_num = 10;
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,02.10.1996


In [61]:
#4.Выберите самый дорогой заказ, выведите список товаров с их ценами.

tasks+=1
pd.read_sql(
    '''
SELECT OD.OrderID, P.ProductID, P.ProductName, OD.Quantity
FROM Products P left join OrderDetails OD on P.ProductID=OD.ProductID
WHERE OD.OrderID = (
  SELECT OrderID
  FROM Products left join OrderDetails on Products.ProductID=OrderDetails.ProductID
  GROUP BY OrderID
  ORDER BY SUM(Price * Quantity) DESC
  LIMIT 1
)
      
    ''',
    con,
)

#Алексей: Самый дорогой заказ нужно определять по сумме цен, помноженных на кол-во единиц товара - sum(Price*Quantity). По полученному OrderID вывести необходимые данные.
#Илья: Скорректировал

,OrderID,ProductID,ProductName,Quantity
0,10372,20,Sir Rodney's Marmalade,12
1,10372,38,Côte de Blaye,40
2,10372,60,Camembert Pierrot,70
3,10372,72,Mozzarella di Giovanni,42


In [62]:
#5.Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

tasks+=1
pd.read_sql(
    '''
     
  SELECT P.ProductID, ProductName, SUM(Quantity) AS TotalQuantity
  FROM Products P left join OrderDetails O on P.ProductID=O.ProductID
  GROUP BY P.ProductID
  ORDER BY SUM(Quantity) DESC
  LIMIT 1
    
    ''',
    con,
)

#Алексей: Нужно найти ProductID товара, у которого было заказано наибольшее кол-во единиц товара и вывести по нему необходимую информацию - название и количество единиц в каждом из заказов.
#Илья: Скорректировал

,ProductID,ProductName,TotalQuantity
0,31,Gorgonzola Telino,458


In [63]:
#6.Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

tasks+=1
pd.read_sql(
    '''
    SELECT SupplierName,  Country, ContactName, Phone
    FROM Suppliers S left join Products P on S.SupplierID=P.SupplierID
    left join OrderDetails OD on P.ProductID=OD.ProductID
    GROUP BY SupplierName
    HAVING  count(distinct OrderID)
    ORDER BY SupplierName desc
    LIMIT 5;
    ''',
    con,
)


#Алексей: Почему сортировка идет по имени поставщика - SupplierName ? Она должна быть по кол-ву заказов - count(distinct OrderID)
#Илья: Скорректировал

,SupplierName,Country,ContactName,Phone
0,Zaanse Snoepfabriek,Netherlands,Dirk Luchte,(12345) 1212
1,Tokyo Traders,Japan,Yoshi Nagase,(03) 3555-5011
2,Svensk Sjöföda AB,Sweden,Michael Björn,08-123 45 67
3,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
4,Refrescos Americanas LTDA,Brazil,Carlos Diaz,(11) 555 4640


In [64]:
#7.Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

tasks+=1
pd.read_sql(
    '''
    SELECT Country, CategoryName, sum(Price*Quantity) as TOTAL_Price
    FROM Categories  left join Products on Categories.CategoryID=Products.CategoryID
    left join OrderDetails on Products.ProductID=OrderDetails.ProductID
    left join Orders on OrderDetails.OrderID=Orders.OrderID
    left join Customers on Orders.CustomerID=Customers.CustomerID
    where Country = 'Brazil'
    GROUP BY CategoryName
    ORDER BY TOTAL_Price desc
    ''',
    con,
)

#Алексей: "ORDER BY Price desc" - определяется по максимальной цене одной единицы товара. Общую сумму нужно определять по сумме цен, помноженных на кол-во единиц товара - sum(Price*Quantity).
#Илья: Скорректировал

,Country,CategoryName,TOTAL_Price
0,Brazil,Beverages,13690.00
1,Brazil,Confections,6307.30
2,Brazil,Produce,6205.45
3,Brazil,Dairy Products,5491.60
4,Brazil,Meat/Poultry,4780.85
5,Brazil,Condiments,2748.95
6,Brazil,Seafood,991.10


In [65]:
#8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

tasks+=1
pd.read_sql(
    '''
   
   WITH OP AS (
    SELECT Orders.OrderID, SUM(OrderDetails.quantity * Products.price) AS total_price
    FROM Categories left join Products on Categories.CategoryID=Products.CategoryID
    left join OrderDetails on Products.ProductID=OrderDetails.ProductID
    left join Orders on OrderDetails.OrderID=Orders.OrderID
    left join Customers on Orders.CustomerID=Customers.CustomerID
    where Country = 'USA'
    GROUP BY Orders.OrderID
),
MaxMinPrices AS (
    SELECT MAX(total_price) AS max_price, MIN(total_price) AS min_price
    FROM OP
)

SELECT (max_price - min_price) AS price_difference
FROM MaxMinPrices;
   
    ''',
    con,
)

,price_difference
0,7638.45


In [66]:
#9.Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

tasks+=1
pd.read_sql(
    '''
SELECT 
  count(o.OrderID) as NumberOfOrders,
  e.FirstName || ' ' || e.LastName AS EmployeeName,
  SUBSTR(e.BirthDate, 9, 2) || '-' || SUBSTR(e.BirthDate, 6, 2) || '-' || SUBSTR(e.BirthDate, 1, 4) as NewBirthDateString
from Employees e join Orders o on e.EmployeeID = o.EmployeeID
where e.EmployeeID in (
  SELECT EmployeeID
  FROM Employees
  ORDER BY BirthDate DESC
  LIMIT 3
)
GROUP BY e.EmployeeID
ORDER BY NewBirthDateString desc;
  
    ''',
    con,
)


#Алексей: Нужно составить новую дату в виде строки(с помощью substr) и определять порядок по этой новой строке.
#Илья: Скорректировал

,NumberOfOrders,EmployeeName,NewBirthDateString
0,31,Janet Leverling,63-.1-30.0
1,14,Robert King,60-.1-29.0
2,40,Margaret Peacock,58-.1-19.0


In [100]:
#10.Сколько банок крабового мяса всего было заказано.

tasks+=1
pd.read_sql(
    '''   
    select (sum*Unit_Crab) as total_crab
    from 
    (SELECT ProductName, sum(Quantity) as sum, substr(Unit, 1, 3) as Unit_Crab
    FROM Products left join OrderDetails on Products.ProductID=OrderDetails.ProductID
    where ProductName = 'Boston Crab Meat')
      
    ''',
    con,
)

# Алексей: Из таблицы Products, колонки Unit нужно было вытащить "24" и его умножить на сумму Quantity ("24 - 4 oz tins" - означает "24 банки по 4 унции").
# Илья: скорректировал

,total_crab
0,6144


In [68]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены не все задания! 🙀
